# Create a Short-Position Algorithm

An algorithm to identify the entry and exit points for a short-position trading strategy. The algorithm will still use a short-window (50 days) SMA and a long-window (100 days) SMA.

In [8]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

## Create a DMAC Trading Signal

### Step 1: Read the `vnq.csv` file from the Resources folder into a Pandas DataFrame. Set the “Date” column as the index. Review the DataFrame.

In [9]:
# Read the vnq.csv file from the Resources folder into a Pandas DataFrame
# Set the `Date` column as the DateTimeIndex
vnq_df = pd.read_csv(Path("Resources/vnq.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)

# Review the DataFrame
vnq_df.head()


,Open,High,Low,Close,Volume
Date,,,,,
2007-01-03,77.339996,77.709999,76.400002,44.707417,525700
2007-01-04,77.230003,77.260002,76.629997,44.765377,166100
2007-01-05,77.099998,77.099998,75.750000,43.936401,143700
2007-01-08,76.000000,76.000000,75.430000,44.000164,87000
2007-01-09,75.730003,76.959999,75.730003,44.556675,136300


## Step 2: By filtering the DataFrame, create a second DataFame, named `signals_df`, that filters out only the “Date” index and the “Close” column. Review the new DataFrame, and then use hvPlot to visualize the pricing information.

In [10]:
# Create a DataFrame filtering only the index and Close columns
signals_df = vnq_df.loc[:,["Close"]]

# Review the DataFrame
signals_df.head()


,Close
Date,
2007-01-03,44.707417
2007-01-04,44.765377
2007-01-05,43.936401
2007-01-08,44.000164
2007-01-09,44.556675


In [11]:
# Use hvplot to visualize the DataFrame
signals_df.hvplot()


:Curve   [Date]   (Close)

### Step 3: Set the `short_window` and `long_window` variables to 50 and 100, respectively.

In [12]:
# Set the short_window (50) and long window (100) variables
short_window = 50
long_window = 100


### Step 4: Using the “Close” columns as well as the `rolling` and `mean` functions, create columns named “SMA50” and “SMA100” that contain the 50-day SMA and 100-day SMA simple moving averages, respectively. Review the DataFrame.

In [13]:
# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df["Close"].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df["Close"].rolling(window=long_window).mean()

# Review the DataFrame
signals_df.tail()


,Close,SMA50,SMA100
Date,,,
2009-12-24,31.367579,28.438977,27.586397
2009-12-28,31.517859,28.504734,27.645874
2009-12-29,31.046522,28.575901,27.702575
2009-12-30,31.094339,28.636155,27.745596
2009-12-31,30.561541,28.696138,27.789032


### Step 5: Create a new column, named “Signal”, in the DataFrame, and set all its values to 0.0.

In [14]:
# Initialize the new Signal column to hold the trading signal
signals_df['Signal'] = 0.0


### Step 6: Using the NumPy `where` function, set the “Signal” column to 1.0 when the “SMA50” value is less than the “SMA100” value. Otherwise, set the “Signal” column to 0.0.

In [15]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) is less than the long-window (SMA100)
signals_df["Signal"][short_window:] = np.where(signals_df['SMA50'][short_window:] < signals_df['SMA100'][short_window:], 1.0, 0.0)

# Review the DataFrame
signals_df.iloc[95:105,:]


,Close,SMA50,SMA100,Signal
Date,,,,
2007-05-21,43.642437,46.194911,NaN,0.0
2007-05-22,44.289658,46.145049,NaN,0.0
2007-05-23,43.951462,46.110452,NaN,0.0
2007-05-24,43.216797,46.056755,NaN,0.0
2007-05-25,43.461689,45.999724,46.646618,1.0
2007-05-29,45.012638,45.984032,46.649670,1.0
2007-05-30,46.219574,45.983318,46.664212,1.0
2007-05-31,46.033009,45.975163,46.685178,1.0
2007-06-01,46.097130,45.957044,46.706148,1.0


### Step 7: Use the Pandas `diff` function on the “Signal” column to assign values of 1 or &minus;1 to the “Entry/Exit” column. These values will indicate the trade entry and exit points in time. Review the DataFrame.

In [16]:
# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.loc["2007-10-20":"2007-10-30",:]

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2007-10-22,42.291416,41.847472,41.930229,1.0,0.0
2007-10-23,42.738773,41.916470,41.896646,0.0,-1.0
2007-10-24,42.022976,41.993704,41.854038,0.0,0.0
2007-10-25,42.112465,42.085598,41.819673,0.0,0.0
2007-10-26,42.750698,42.160148,41.794779,0.0,0.0
2007-10-29,42.052799,42.203029,41.777134,0.0,0.0
2007-10-30,42.631397,42.242841,41.760026,0.0,0.0


## Plot the Entry and Exit Points of the Trading Signal

### Step 1: Using hvPlot, create two scatter plots: one for the entry points and another for the exit points. Use purple markers to indicate the entry points (when the algorithm sells the stock). Use orange markers to indicate the exit points (when the algorithm buys the stock to cover the short position).

  > **Hint:** Remember that you’re creating a short-position algorithm. The signal to trade (1) occurs when the short-window SMA (the “SMA50” value) is less than the long-window SMA (the “SMA100” value).


In [17]:
# Visualize entry positions relative to close price
entry = signals_df[signals_df['Entry/Exit']==1.0]["Close"].hvplot.scatter(
    color="purple", 
    marker="^", 
    legend=False, 
    ylabel='Price in $', 
    width=1000, 
    height=400
)
entry

:Scatter   [Date]   (Close)

In [18]:
# Visualize exit positions relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color="orange", 
    marker='v', 
    legend=False, 
    ylabel='Price in $', 
    width=1000, 
    height=400
)
exit

:Scatter   [Date]   (Close)

In [19]:
# Plot the entry and exit points
entry * exit


:Overlay
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

### Step 2: Create two line plots: one for the VNQ closing prices and another for the 50-day and 100-day SMAs.

In [20]:
# Visualize Close price for the investment
security_close = signals_df[["Close"]].hvplot(
    line_color="lightgray", 
    ylabel='Price in $', 
    width=1000, 
    height=400
)

# Visualize the SMA50 and SMA 100 moving averages
moving_avgs = signals_df[["SMA50", "SMA100"]].hvplot(
    ylabel='Price in $', 
    width=1000, 
    height=400, 
    line_color=["skyblue", "red"]
)


In [23]:
# Plot the security_close and moving_avgs
security_close * moving_avgs


:Overlay
   .Curve.I     :Curve   [Date]   (Close)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)

### Step 3: Combine all four plots from Steps 1 and 2 into one overlay plot.

In [22]:
# Overlay all four plots in a single visualization
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(
    title="VNQ - Short-Position Dual Moving Average Trading Algorithm"
)


:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)